In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from Data import*

bom_df = pd.read_excel("Bill of Material (mrp.bom)updated.xlsx")
stock_df = pd.read_excel("Reordering Rule.xlsx")

Stockproducts = list(stock_df['Product'])
#Machines_df = pd.read_excel("Maintenance Equipment.xlsx")
#Machines =list(Machines_df['']
Machines = ['M3-01 - Accuwell - 3axis - Manual (FR3_01)','M5-03 - Haas - 5axis - Manual (UMC750)','M3-02 - Accuwell - 3axis - Manual (FR3_02)','M3-03 - Haas - 3axis - Manual (VF2SS)']
AutomatedMachines = ['M5-04 - Haas - 5axis  - Manual (UMC750SS)','M5-01 - Accuwell - 5axis - Pallet automaton (FR5_01)','M4-01 - Accuwell -  4axis - Conveyor automation (FR4_01)','M4-02 - Accuwell - 4axis - Conveyor automation (FR4_02)','M5-02 - Accuwell - 5axis - Conveyor automation (FR5_02)']
Manual = ['MP3 - Material preparation - pre-cutted, only picking (UITG_M)','MP2 - Material preparation - manual sawing (ZGN_02)','BW - Benchwork (BKW_01)','PACK - Packaging (VERP_P)','MP1 - Material preparation - automatic sawing (ZGN_01)']

DataMgr = DataManager()
DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct (dbmgr,product,stock):

    if (product.find("[")>-1) and (product.find("]")>-1):
        productnumber = product[product.find("[")+1:product.find("]")];
        productname = product[product.find("]")+1:];
        if not "["+productnumber+"] "+productname in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),"["+productnumber+"] "+productname,productnumber,stock);
            return newproduct
        else:
            return dbmgr.getProducts()["["+productnumber+"] "+productname]
    else:
        if not product in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),product,product,stock);
            return newproduct
        else:
            return dbmgr.getProducts()[product]

def MakeOperation(dbmgr,operation,prod,resource,duration):
    myresource=None;
    if not resource in dbmgr.getResources():
        if resource in Machines or resource in AutomatedMachines:
            if resource in Machines:
              myresource = Resource(len(dbmgr.getResources()),"Machine",resource,15);
              myresource.setAutomated(False);
              myresource.setOperatingEffort(0.3);
              myresource.setAvailableShift("All");
              dbmgr.getResources()[resource]=myresource;
            else:
               myresource = Resource(len(dbmgr.getResources()),"Machine",resource,24);
               dbmgr.getResources()[resource]=myresource;
               myresource.setAutomated(True);
               myresource.setAvailableShift("All");
               myresource.setOperatingEffort(0.1);
        elif "Outsourced" in resource.split():
            myresource = Resource(len(dbmgr.getResources()),"Outsourced",resource,10000);
            myresource.setAvailableShift("All");
            dbmgr.getResources()[resource]=myresource;
        else:
            myresource = Resource(len(dbmgr.getResources()),"Manual",resource,15);
            if resource == 'MP1 - Material preparation - automatic sawing (ZGN_01)':
                myresource.setOperatingEffort(0.15)
            elif resource == 'MP3 - Material preparation - pre-cutted, only picking (UITG_M)':
                myresource.setOperatingEffort(0)
            else:                
                myresource.setOperatingEffort(1);
            myresource.setAvailableShift("All");
            dbmgr.getResources()[resource]=myresource;
    else:
        myresource = dbmgr.getResources()[resource]; 

    
    newopr = operation

    if myresource !=None:
        myresource.getOperations().append(newopr)
        newopr.getRequiredResources().append(myresource)

    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()]=newopr;
        
    return newopr

def SaveInstance(DataMgr,file):

        # Save Products.
        products_df = pd.DataFrame(columns= ["ProductID", "ProductNumber","Name","StockUnit","StockBatch","StockLevel","PrescribedBatchsize","ChosenBatchsize"])
        precedences_df = pd.DataFrame(columns= ["PredecessorID","SuccessorID","Multiplier"])
        prodrops_df = pd.DataFrame(columns= ["ProductID","OperationID","OperationIndex"])
        operations_df = pd.DataFrame(columns= ["OperationID","Name","ProcessTime","Predecessor"])
        opsres_df = pd.DataFrame(columns= ["OperationID","ResourceID"])
        resources_df = pd.DataFrame(columns= ["ResourceID","ResourceType","Name","DailyCapacity","Automated","OperatingEffort","Shift"])
        orders_df = pd.DataFrame(columns= ["OrderID","ProductID","ProductName","Name","Quantity","Deadline"])
 
        
        for name,myprod in DataMgr.Products.items():
            products_df.loc[len(products_df)] = {"ProductID":myprod.getID(), "ProductNumber":myprod.getPN(),"Name":myprod.getName(),"StockUnit":myprod.getStockUnit(),"StockBatch":myprod.getStockBatch(),"StockLevel":myprod.getStockLevel(),"PrescribedBatchsize":myprod.getPrescribedBatchsize(),"ChosenBatchsize":myprod.getChosenBatchsize()}

            
            for pred in myprod.getPredecessors():
                precedences_df.loc[len(precedences_df)] = {"PredecessorID":pred.getID(),"SuccessorID":myprod.getID(),"Multiplier":myprod.getMPredecessors()[pred]}
 
            oprind = 0
            for opr in myprod.getOperations(): 
                prodrops_df.loc[len(prodrops_df)] = {"ProductID":myprod.getID(),"OperationID":opr.getID(),"OperationIndex":oprind}
                oprind+=1 

        for opname,opr in DataMgr.Operations.items():
            pred = ""
            if list(opr.getPredecessor().keys()) != []:
                pred = list(opr.getPredecessor().keys())[0]
            operations_df.loc[len(operations_df)]= {"OperationID":opr.getID(),"Name":opr.getName(),"ProcessTime":opr.getProcessTime(),"Predecessor":pred}
            
            for res in opr.getRequiredResources(): 
                opsres_df.loc[len(opsres_df)]= {"OperationID":opr.getID(),"ResourceID":res.getID()}
        

        for resname,res in DataMgr.Resources.items():
            resources_df.loc[len(resources_df)] ={"ResourceID":res.getID(),"ResourceType":res.getType(),"Name":res.getName(),"DailyCapacity":res.getDailyCapacity(),"Automated":res.getAutomated(),"OperatingEffort":res.getOperatingEffort(),"Shift":res.getAvailableShift()}
 
        
        for ordname,ordr in DataMgr.CustomerOrders.items():
            orders_df.loc[len(orders_df)]={"OrderID":ordr.getID(),"ProductID":ordr.getProduct().getID(),"ProductName":ordr.getProduct().getName(),"Name":ordr.getName(),"Quantity":ordr.getQuantity(),"Deadline":ordr.getDeadLine()}

    


        folder = 'UseCases'; casename = file
        path = folder+"\\"+casename
        isExist = os.path.exists(path)

        if not isExist:
            os.makedirs(path)
       
        
        filename = 'Products.csv'; path = folder+"\\"+casename+"\\"+filename;  fullpath = os.path.join(Path.cwd(), path) 
        products_df.to_csv(path, index=False)
        filename = 'Precedences.csv';path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        precedences_df.to_csv(fullpath, index=False)
        filename = 'ProductsOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        prodrops_df.to_csv(fullpath, index=False)
        filename = 'ResourcesOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        opsres_df.to_csv(fullpath, index=False)
        filename = 'Operations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        operations_df.to_csv(fullpath, index=False)
        filename = 'Resources.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        resources_df.to_csv(fullpath, index=False)
        filename = 'CustomerOrders.csv'; path = folder+"\\"+casename+"\\"+filename;fullpath = os.path.join(Path.cwd(), path)
        orders_df.to_csv(fullpath, index=False)

        return

##Create operators and manual worker
Op1 = Resource(len(DataMgr.getResources()),"Operator","Operator 1",8);
Op1.setAvailableShift(1);
DataMgr.getResources()["Operator 1"]=Op1;

Op2 = Resource(len(DataMgr.getResources()),"Operator","Operator 2",8);
Op2.setAvailableShift(1);
DataMgr.getResources()["Operator 2"]=Op2;

Op3 = Resource(len(DataMgr.getResources()),"Operator","Operator 3",7);
Op3.setAvailableShift(2);
DataMgr.getResources()["Operator 3"]=Op3;

ManWor = Resource(len(DataMgr.getResources()),"Operator","Manual workers",15);
DataMgr.getResources()["Manual workers"]=ManWor;
##End create operators

PrescribedBatches=dict()

for row, rule in stock_df.iterrows():
    PrescribedBatches[rule['Product']]=rule['Multiple Quantity']



nopass = 0;
Stockproducts = list(stock_df['Product'])
for i,r in bom_df.iterrows():
    CreatedOperations=[]
    productrow = False;
    if nopass > 0:
        nopass -=1
        continue

    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow = True;

    if productrow == True:
        newproduct = MakeProduct(DataMgr,r['Product'],0)
        if r['Product'] in PrescribedBatches:
            newproduct.setPrescribedBatchsize(PrescribedBatches[r['Product']])
        newproduct.setStockUnit(r['BoM Lines/Product Unit of Measure'])
        if r['BoM Lines/Product Unit of Measure'] == 'mm':
            newproduct.setStockBatch(3000)
        DataMgr.getProducts()[newproduct.getName()] = newproduct;

        if not pd.isnull(r['BoM Lines/Component']):
            rawprod=r['BoM Lines/Component']
            if rawprod in Stockproducts:
                stock = stock_df['On Hand'][Stockproducts.index(rawprod)]
            else:
                stock=0

            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'],stock)
            
            quantity = r['BoM Lines/Quantity']

            newproduct.getMPredecessors()[rawprod] = quantity
            newproduct.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newproduct)
            if r['Product'] in PrescribedBatches:
                rawprod.setPrescribedBatchsize(PrescribedBatches[r['Product']])

            DataMgr.getProducts()[rawprod.getName()] = rawprod

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])
            duration = bom_df['BoM Lines/Operations/Duration'][j]
            if isinstance(duration, (int, float)):
                dur = float(1/60)*float(duration) # time assumed to be minute..

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                if bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')] == "OUT - Outsourced activity":
                    myopr = Operation(len(DataMgr.getOperations()),bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]+"_"+newproduct.getName(),dur)
                    myopr = MakeOperation(DataMgr,myopr,newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')]+"_"+bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],dur)
                    newproduct.getOperations().append(myopr)                    
                    if j > i:
                        myopr.setPredecessor(CreatedOperations[-1])
                    CreatedOperations.append(myopr)
                else:
                    myopr = Operation(len(DataMgr.getOperations()),bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]+"_"+newproduct.getName(),dur)
                    myopr = MakeOperation(DataMgr,myopr,newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')],dur)
                    newproduct.getOperations().append(myopr.getName())
                    if j > i:
                        myopr.setPredecessor(CreatedOperations[-1])
                    CreatedOperations.append(myopr)
               
   
            j+=1 
        nopass=len(newproduct.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)

rel_path = "Sales Order Line (sale.order.line).xlsx"
abs_file_path = os.path.join(Path.cwd(), rel_path)
xls = pd.ExcelFile(abs_file_path)
orders_df = pd.read_excel(xls,"Sheet1")

#orders_df['Delivery Date']  = pd.to_datetime(orders_df['Delivery Date'] )

for i,r in orders_df.iterrows():
    code = r['Description']
    if code.find("[")>-1 and code.find("]")>-1:
        code = code[code.find("[")+1:code.find("]")]
    else:
        continue
    
    ordername = r['Order Reference']+"_("+code+")"

    prodmatch = [prod for prod in DataMgr.getProducts().values() if prod.getPN() == code]

    if len(prodmatch) == 0:
        continue

    myprod = prodmatch[0]
    if myprod.getPrescribedBatchsize() is not None:
        myprod.setChosenBatchsize(myprod.getPrescribedBatchsize())
    else:
        myprod.setChosenBatchsize(r['Quantity']) 

    deadline = datetime.today().strftime('%Y-%m-%d')

    if not pd.isnull(r['Delivery Date']):
        deadline = r['Delivery Date'].strftime('%Y-%m-%d')
    

    #print(r['Delivery Date'],pd.isnull(r['Delivery Date']))
    myorder = CustomerOrder(len(DataMgr.getCustomerOrders()),ordername,myprod.getID()
                                                           ,myprod.getName(),r['Quantity'],str(deadline))
    myorder.setProduct(myprod)

    DataMgr.getCustomerOrders()[ordername] = myorder

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))
print("Nr Customer Orders: ",len(DataMgr.getCustomerOrders()))

# SaveInstance(DataMgr,'TBRM_Volledige_Instantie')

    

    
    
        

    

Nr Products:  561
Nr Resources:  33
Nr Operations:  984
Nr Customer Orders:  127


In [1]:
import os
online_version = False;
from DBMain import*

#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); tab_2 = VisMgr.generateCOTAB(); tab_3 = VisMgr.generatePLTAB()
tab_5 = VisMgr.getSchedulingTab().generatePSschTAB()
###################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_2,tab_3,tab_5])
tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Production System'); tab_set.set_title(2, 'Customer Orders');
tab_set.set_title(3, 'Production Planning'); tab_set.set_title(4, 'Production Schedule');tab_set

In [2]:
test = DataMgr


In [ ]:
oprdict = dict() # key: operation, #val: set of jobs
      

for resname,res in self.getDataManager().getResources().items():
    res.getSchedule().clear()

# Collect operations with jobs
nrjobs = 0
for prname,prod in self.getDataManager().getProducts().items():
    for opr in prod.getOperations():
        if not opr in oprdict:
            oprdict[opr] = opr.getJobs()
            nrjobs+= len(opr.getJobs())
            
self.getVisualManager().getPSchScheRes().value+=" To schedule jobs: "+str(nrjobs)+"\n"
revopdict = {k: oprdict[k] for k in sorted(oprdict, key=lambda x: list(oprdict.keys()).index(x), reverse=True)}


for k1, k2 in zip(revopdict, list(revopdict)[1:]): #links pairs of keys together 
    if len(revopdict[k1]) > 0 and len(revopdict[k2]) >0:
        CurJobs = revopdict[k1][::-1];
        Predjobs = revopdict[k2][::-1];
        
        for i in range(0,len(CurJobs)):
            CapCurJob = CurJobs[i].getQuantity();           
                      
            
            for k in Predjobs:
                
                if k.getQuantity() < CapCurJob:
                    CapCurJob = CapCurJob - k.getQuantity();
                    if k not in CurJobs[i].getPredecessors():
                        CurJobs[i].getPredecessors().append(k);
                    if CurJobs[i] not in k.getSuccessor():
                        k.getSuccessor().append(CurJobs[i]);
                    Predjobs.remove(k)
                else:
                    if k not in CurJobs[i].getPredecessors():
                        CurJobs[i].getPredecessors().append(k);
                    
                    if CurJobs[i] not in k.getSuccessor():
                        k.getSuccessor().append(CurJobs[i]);
                    Predjobs.remove(k)                       
                    break
## Initialize shifts (example 30 days?)
shiftnum = 1000;

i=1;
day=1;
shiftlist=[]
while i <= shiftnum:
    if i % 2 == 1:
        shift = Shift(i,1,8)
        shiftlist.append(shift)
        i+=1;
    else:
        shift=Shift(i,2,7)
        shiftlist.append(shift)
        i+=1

#Initialize Schedulable Jobs
AllJobs = dict()
SchedulableJobs=[]

ScheduledJobs=[]
for opr, jobs in oprdict.items():
    for job in jobs:
        if job.getPredecessors() == []:
            SchedulableJobs.append(job)
        AllJobs[job.getName()]=job;
counter = 0       
#Initialize schedule for each Resource:
for resname, res in self.getDataManager().getResources().items():
    for i in shiftlist:
        res.getSchedule()[i]=[]

#Create Schedule
while len(SchedulableJobs) >0:
    for j in SchedulableJobs:
        predecessorjobs = j.getPredecessors()
        successorjobs = j.getSuccessor()
        if not (predecessorjobs ==[]):
        #Determine Earliest starttime
            maxpredecessor = None;
            for i in predecessorjobs:
                completiontime = i.getStartTime() + (i.getQuantity() * self.getDataManager().getOperations()[i.getOperation().getName()].getProcessTime())
                if maxpredecessor == None or completiontime >= maxpredecessor:
                    maxpredecessor = completiontime
            EarliestStart = maxpredecessor;
        else:
            EarliestStart = 0
            
        #Determine earliest available resource
        processtime = (j.getQuantity() * self.getDataManager().getOperations()[j.getOperation().getName()].getProcessTime())
        resources = j.getOperation().getRequiredResources()
        for r in resources:
            Automated = r.getAutomated()
            opef = r.getOperatingEffort()
            for shift, jobtime in r.getSchedule().items():
                
                ##Here we now have the completion time of the last job in a shift. Check if the to job to schedule fits in the shift. Else check next shift.
                if jobtime ==[]:
                    completiontimeLatestJob = 0;
                else:
                    completiontimeLatestJob = jobtime[::-1][0][1] + (jobtime[::-1][0][0].getQuantity() * self.getDataManager().getOperations()[jobtime[::-1][0][0].getOperation().getName()].getProcessTime())
                
                shiftcap = shift.getCapacity()
                shiftnumber = shift.getNumber()
                
                if shiftnumber == 1:
                    CurOpeffOp1 = 0; ## Also check operator availability.
                    SchedOp1 = self.getDataManager().getResources()['Operator 1'].getSchedule();
                    CurOpeffOp2 = 0;
                    SchedOp2 = self.getDataManager().getResources()['Operator 2'].getSchedule();
                    for jobs in SchedOp1[shift]:
                        for resOpJob in jobs[0].getOperation().getRequiredResources():
                            CurOpeffOp1 += resOpJob.getOperatingEffort()
                    for jobs in SchedOp2[shift]:
                        for resOpJob in jobs[0].getRequiredResources():
                            CurOpeffOp2 += resOpJob.getOperatingEffort()

                    if shiftcap > (completiontimeLatestJob + processtime) and (CurOpeffOp1 + opef <= 1 or CurOpeffOp2 + opef <=1): #This means we can schedule in first shift
                        j.setStartTime(completiontimeLatestJob)
                        if (CurOpeffOp1 +opef <= 1):
                           SchedOp1[shift].append((j,completiontimeLatestJob))
                        else:
                            SchedOp2[shift].append((j,completiontimeLatestJob))
                        r.getSchedule()[shift].append((j,completiontimeLatestJob))                                
                        ScheduledJobs.append(j)
                        ## check if successor can be scheduled.
                        for sucjob in successorjobs:
                            Schedulable = True
                            for predjbs in sucjob.getPredecessors():
                                if predjbs in ScheduledJobs:
                                    continue
                                else:
                                    Schedulable = False;
                            if Schedulable == True:
                                SchedulableJobs.append(sucjob)
                        
                        SchedulableJobs.remove(j) #Remove scheduled job
                        break
                    else:
                        continue

                if shiftnumber == 2:
                    CurOpeffOp3 = 0; ## Also check operator availability.
                    SchedOp3 = self.getDataManager().getResources()['Operator 3'].getSchedule();
                    
                    for jobs in SchedOp3[shift]:
                        for resOpJob in jobs[0].getOperation().getRequiredResources():
                            CurOpeffOp3 += resOpJob.getOperatingEffort()
                    if  ((Automated is None) and processtime < 8) or Automated == False:
                        if shiftcap > (completiontimeLatestJob + processtime) and (CurOpeffOp3 + opef<=1): #This means we can schedule in second shift
                            j.setStartTime(completiontimeLatestJob)
                            SchedOp3[shift].append((j,completiontimeLatestJob))
                            
                            r.getSchedule()[shift].append((j,completiontimeLatestJob))                                
                            ScheduledJobs.append(j)
                            ## check if successors can be scheduled.
                            for sucjob in successorjobs:
                                Schedulable = True
                                for predjbs in sucjob.getPredecessors():
                                    if predjbs in ScheduledJobs:
                                        continue
                                    else:
                                        Schedulable = False;
                                if Schedulable == True:
                                    SchedulableJobs.append(sucjob)
                            
                            SchedulableJobs.remove(j) #Remove scheduled job
                            break
                        else:
                            continue

                    elif (Automated == True) or (Automated is None and processtime > 8):                        
                        if shiftcap >= (completiontimeLatestJob) and (CurOpeffOp3 + opef<=1): #This means we can schedule in second shift
                            j.setStartTime(completiontimeLatestJob)
                            SchedOp3[shift].append((j,completiontimeLatestJob))
                            
                            r.getSchedule()[shift].append((j,completiontimeLatestJob))                                
                            ScheduledJobs.append(j)
                            ## check if successors can be scheduled.
                            for sucjob in successorjobs:
                                Schedulable = True
                                for predjbs in sucjob.getPredecessors():
                                    if predjbs in ScheduledJobs:
                                        continue
                                    else:
                                        Schedulable = False;
                                if Schedulable == True:
                                    SchedulableJobs.append(sucjob)
                            
                            SchedulableJobs.remove(j) #Remove scheduled job
                            break
                        else:
                            continue                        
            break
            

In [3]:
schedules_df = pd.DataFrame(columns= ["ResourceID","ResourceName","Shift","JobID","Starttime","Day"])

In [4]:
casename = 'TBRM_Volledige_Instantie'
folder = 'UseCases'; casename
path = folder+"\\"+casename
isExist = os.path.exists(path)

if not isExist:
    os.makedirs(path)

for name,myres in DataMgr.Resources.items():
    
    if name != 'Operator 1' and name != 'Operator 2' and name != 'Operator 3' and myres.getName() != 'Manual workers':
        for shift, jobs in myres.getSchedule().items():
            if jobs == []:
                continue
            else:                
                for job in jobs:
                    schedules_df.loc[len(schedules_df)] = {"ResourceID":myres.getID(),"ResourceName":name, "Shift":shift.getNumber(),"JobID":job[0].getID(),"Starttime":str(job[1]),"Day":shift.getDay()}
            
filename = 'Schedules.csv'; path = folder+"\\"+casename+"\\"+filename;fullpath = os.path.join(Path.cwd(), path)
schedules_df.to_csv(fullpath, index=False)

Operator 1
Operator 2
Operator 3
Manual workers
MP2 - Material preparation - manual sawing (ZGN_02)
M3-01 - Accuwell - 3axis - Manual (FR3_01)
BW - Benchwork (BKW_01)
PACK - Packaging (VERP_P)
OUT - Outsourced activity_Chromising
MP1 - Material preparation - automatic sawing (ZGN_01)
M5-01 - Accuwell - 5axis - Pallet automaton (FR5_01)
M4-01 - Accuwell -  4axis - Conveyor automation (FR4_01)
M5-03 - Haas - 5axis - Manual (UMC750)
M3-02 - Accuwell - 3axis - Manual (FR3_02)
OUT - Outsourced activity_Anodising
OUT - Outsourced activity_Painting
MP3 - Material preparation - pre-cutted, only picking (UITG_M)
M4-02 - Accuwell - 4axis - Conveyor automation (FR4_02)
OUT - Outsourced activity_Passivating
OUT - Outsourced activity_Cleaning
OUT - Outsourced activity_Sandblasting
OUT - Outsourced activity_Nickel plating
M3-03 - Haas - 3axis - Manual (VF2SS)
M5-02 - Accuwell - 5axis - Conveyor automation (FR5_02)
OUT - Outsourced activity_Anodising + Cleaning
OUT - Outsourced activity_Anodize
OUT -

In [6]:
oprdict = dict() # key: operation, #val: set of jobs
      

for resname,res in DataMgr.getResources().items():
    res.getSchedule().clear()

# Collect operations with jobs
nrjobs = 0
for prname,prod in DataMgr.getProducts().items():
    for opr in prod.getOperations():
        if not opr in oprdict:
            oprdict[opr] = opr.getJobs()
            nrjobs+= len(opr.getJobs())
            

revopdict = {k: oprdict[k] for k in sorted(oprdict, key=lambda x: list(oprdict.keys()).index(x), reverse=True)}


for k1, k2 in zip(revopdict, list(revopdict)[1:]): #links pairs of keys together 
    if len(revopdict[k1]) > 0 and len(revopdict[k2]) >0:
        CurJobs = revopdict[k1][::-1];
        Predjobs = revopdict[k2][::-1];
        
        for i in range(0,len(CurJobs)):
            CapCurJob = CurJobs[i].getQuantity();           
                      
            
            for k in Predjobs:
                
                if k.getQuantity() < CapCurJob:
                    CapCurJob = CapCurJob - k.getQuantity();
                    if k not in CurJobs[i].getPredecessors():
                        CurJobs[i].getPredecessors().append(k);
                    if CurJobs[i] not in k.getSuccessor():
                        k.getSuccessor().append(CurJobs[i]);
                    Predjobs.remove(k)
                else:
                    if k not in CurJobs[i].getPredecessors():
                        CurJobs[i].getPredecessors().append(k);
                    
                    if CurJobs[i] not in k.getSuccessor():
                        k.getSuccessor().append(CurJobs[i]);
                    Predjobs.remove(k)                       
                    break

In [ ]:
oprdict

In [ ]:
revopdict

In [13]:
test.getProducts()['[00236-05-103-C]  geleideblok spanrol'].getPredecessors()[0].getQuantity()

AttributeError: 'Product' object has no attribute 'getQuantity'

In [7]:
test.getCustomerOrders()['S02854_(00236-05-103-C)'].getProduct().getStockLevel() = test.getCustomerOrders()['S02854_(00236-05-103-C)'].getProduct().getStockLevel() +5


SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (373504330.py, line 1)

In [ ]:
check

In [13]:
check2=dict()

In [14]:
for n in check:
    check2[n] = check2.get(n,0) + 1

In [ ]:
check2

In [16]:
for a,b in check2.items():
    if b>1:
        print('hoi')

In [ ]:
oprdict

In [18]:
oprdict[<PlanningObjects.Operation at 0x218918eff20>]

SyntaxError: invalid syntax (4198264705.py, line 1)

In [19]:
oprdict[PlanningObjects.Operation at 0x218918eff20]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (141169404.py, line 1)

In [ ]:
oprdict.keys()

In [24]:
a = list(oprdict.keys())

In [25]:
oprdict[a[0]]

[]

In [35]:
oprdict[a[5]][7].getPredecessors()

[]

In [ ]:
for i in a:
    for k in range(0,len(oprdict[i])):
        if oprdict[i] == []:
            continue
        if oprdict[i][k].getPredecessors() != []:
            print(oprdict[i][k].getName())

In [ ]:
a

In [3]:
test

In [4]:
test.getVisualManager()

In [5]:
test.getVisualManager().DataManager

In [3]:
test.getVisualManager().DataManager.getProducts()['[BD50-5075-011-E]  bracket'].getOperations()

In [10]:
a = test.getVisualManager().DataManager.getProducts()['[BD50-5075-011-E]  bracket'].getOperations()

In [19]:
for op in a:
    for b in op.getJobs():
        print("hoi")
        

In [15]:
op

In [18]:
op.getJobs()

[]

In [ ]:
test.getResources()


In [ ]:
test.getResources()['M3-01 - Accuwell - 3axis - Manual (FR3_01)'].getSchedule()

In [3]:
a = dict()

In [4]:
a.keys()

dict_keys([])

In [5]:
list(a.keys())

[]